### Part d): Classification  analysis using neural networks

With a well-written code it should now be easy to change the
activation function for the output layer.

Here we will change the cost function for our neural network code
developed in parts b) and c) in order to perform a classification analysis. 

We will here study the Wisconsin Breast Cancer  data set. This is a typical binary classification problem with just one single output, either True or Fale, $0$ or $1$ etc.
You find more information about this at the [Scikit-Learn
site](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html) or at the [University of California
at Irvine](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)). 

To measure the performance of our classification problem we use the
so-called *accuracy* score.  The accuracy is as you would expect just
the number of correctly guessed targets $t_i$ divided by the total
number of targets, that is

$$
\text{Accuracy} = \frac{\sum_{i=1}^n I(t_i = y_i)}{n} ,
$$

where $I$ is the indicator function, $1$ if $t_i = y_i$ and $0$
otherwise if we have a binary classification problem. Here $t_i$
represents the target and $y_i$ the outputs of your FFNN code and $n$ is simply the number of targets $t_i$.

Discuss your results and give a critical analysis of the various parameters, including hyper-parameters like the learning rates and the regularization parameter $\lambda$ (as you did in Ridge Regression), various activation functions, number of hidden layers and nodes and activation functions.  

As stated in the introduction, it can also be useful to study other
datasets. 

Again, we strongly recommend that you compare your own neural Network
code for classification and pertinent results against a similar code using **Scikit-Learn**  or **tensorflow/keras** or **pytorch**.

PLAN:
1. last ned breast cancer data set
2. bruk ffnn til klassifikasjon (med ... som siste lag)
3. Bruk back propogation for å forbedre svaret
4. bruke accuracy til å teste resultatene mine 

In [19]:
## just downloading the dataset
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import autograd.numpy as np
from autograd import grad, elementwise_grad
import FFNN as fn

wisconsin = load_breast_cancer()
X = wisconsin.data
target = wisconsin.target
target = target.reshape(target.shape[0], 1)

#X_train, X_val, t_train, t_val = train_test_split(X, target)

#scaler = MinMaxScaler()
#scaler.fit(X_train)
#X_train = scaler.transform(X_train)
#X_val = scaler.transform(X_val)
print(type(X))
print(X[0])
print(X[0].size) #men sånn egentlig, hva er størrelsen til input-laget?
print(type(target))
print(target.size)
print(target)
## target er en liste med nuller og enere, men alle elementene er lister i seg selv
## altså en slags 569x1 matrise.
## target er svaret på om en pasient har kreft eller ikke. 

""""
Jeg tror problemet mitt er at jeg ikke bruker dataen på riktig måte. 
"""



<class 'numpy.ndarray'>
[1.799e+01 1.038e+01 1.228e+02 1.001e+03 1.184e-01 2.776e-01 3.001e-01
 1.471e-01 2.419e-01 7.871e-02 1.095e+00 9.053e-01 8.589e+00 1.534e+02
 6.399e-03 4.904e-02 5.373e-02 1.587e-02 3.003e-02 6.193e-03 2.538e+01
 1.733e+01 1.846e+02 2.019e+03 1.622e-01 6.656e-01 7.119e-01 2.654e-01
 4.601e-01 1.189e-01]
30
<class 'numpy.ndarray'>
569
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]

'"\nJeg tror problemet mitt er at jeg ikke bruker dataen på riktig måte. \n'

In [20]:
#""""
network_input_size = X[0].size
print(len(X))
#print(X[len(X)-1].size)
#print(X[0].size)
#layer_output_sizes = [X[len(X)-1].size, 3, X[len(X)-1].size]
layer_output_sizes = [1] #er et klassifiseringsproblem, siste lag skal kanskje være 2?
activation_funcs = [fn.ReLU]
activation_ders = [fn.ReLU_der]

layers = fn.create_layers_batch(network_input_size, layer_output_sizes)
batched_layers = [(layers[0][0].T, layers[0][1])]
##fn.create_layers(network_input_size, layer_output_sizes)

predict = fn.feed_forward_batch(X, layers, activation_funcs) ## RunTimeWarning (for høye tall i dataen..)

layer_grads = fn.backpropagation_batch(X, layers, activation_funcs, target, activation_ders) # RunTimeWarning (for høye tall i dataen..)
# ValueError: operands could not be broadcast together with shapes (30,569) (569,30)  
# Tror at shapes må være (x,y)(x,y)?
# problemet er størrelsen på siste laget. Could not broadcast shapes together 

print(layer_grads)

cost_grad = grad(fn.cost, 0)
print(cost_grad(layers, X, activation_funcs, target))

cost_grad = grad(fn.cost_batch, 0)
print(cost_grad(batched_layers, X, activation_funcs, target))
#"""

569


MemoryError: Unable to allocate 41.2 GiB for an array with shape (17070, 323761) and data type float64